<a href="https://colab.research.google.com/github/junoru/Dacon-Competition-01/blob/master/Mission_13_2019_Jeju_BigData_Competition_%ED%87%B4%EA%B7%BC%EC%8B%9C%EA%B0%84_%EB%B2%84%EC%8A%A4%EC%8A%B9%EC%B0%A8%EC%9D%B8%EC%9B%90_%EC%98%88%EC%B8%A1_%EC%B4%88%EA%B8%89%EC%9E%90%EC%9A%A9_%EC%BD%94%EB%93%9C_%EB%94%B0%EB%9D%BC%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **1. 초급자용 코드 따라하기** 

In [54]:
#라이브러리 임포트
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
!pip install geopy

In [0]:
#데이터 받아오기
train = pd.read_csv("/content/drive/My Drive/데이콘 공모전/데이터/13회 제주테크노파크 버스정류장 승하차 예측 data/train.csv")
test = pd.read_csv("/content/drive/My Drive/데이콘 공모전/데이터/13회 제주테크노파크 버스정류장 승하차 예측 data/test.csv")

In [10]:
#데이터 형태 보기
print("train shape : " , train.shape)
print("test shape : ", test.shape)

train shape :  (415423, 21)
test shape :  (228170, 20)


In [17]:
train.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,4.0,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,6.0,26.0,14.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0
4,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


트레인 데이터는 약 41만개, 테스트는 약 23만
주어진 컬럼은 20개, 예측해야 하는 것은 인원.

#1. date 변수 변환

In [14]:
# date 컬럼의 타입 확인
print(type(train['date'][0]))

<class 'str'>


In [0]:
#datetime 형식으로 타입을 변환
train['date'] = pd.to_datetime(train['date'])

In [16]:
# date 컬럼의 타입 변환 확인
print(type(train['date'][0]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [0]:
# 날짜를 명목척도로 사용하기 위해서 숫자를 "월화수목금토일"로 구분
train['weekday'] = train['date'].dt.weekday
train[['weekday']].head()

# 명목변수로 구분된 'date'를 더미/인식 변수, 그러니까 월요일이면 1, 아니면 0인 형태로 바꿔줌
train = pd.get_dummies(train, columns=['weekday'])

In [0]:
# 테스트에 대해서도 똑같이 수행

test['date'] = pd.to_datetime(test['date'])
test['weekday'] = test['date'].dt.weekday
test = pd.get_dummies(test, columns=['weekday'])

In [30]:
# 칼럼 수 확인
train.shape

(415423, 28)

각각의 요일에 해당하는 7개의 피쳐가 늘었음을 확인 할 수 있음.

#2. in_out 변수 확인

In [32]:
train['in_out'].value_counts()

시내    408500
시외      6923
Name: in_out, dtype: int64

시외가 상대적으로 매우 적으므로, 시외를 1 시내를 0으로 변환한다. 
**"이상치"를 1로 놓는 것으로 생각된다.

In [0]:
# train
train['in_out'] = train['in_out'].map({'시내':0, '시외':1})

#x test

test['in_out'] = test['in_out'].map({'시내':0, '시외':1})

#3. **좌표 데이터를 이용한 피쳐 엔지니어링**


제주도 인구가 서귀포시와 제주시에 몰려 있다.그러므로 해당 지역과의 거리를 각각 feature로 추가하겠다. 

* 이 피쳐로 인하여 정확도가 상승할까? 이 피쳐는 탑승 인원 산정에 영향을 주나? 차라리 각 지역별 인구수를 가져오는 편이 더 낫지 않을까?


dis_jejusi : 버스정류장과 제주시와의 거리

dis_seoquipo : 버스정류장과 서귀포시와의 거리

제주시의 위.경도 : 33.500770, 126.522761

서귀포시의 위.경도 : 33.259429, 126.558217

In [0]:
import geopy.distance

coords_jejusi = (33.500770, 126.522761) #제주시의 위도 경도
coords_seoquipo = (33.259429, 126.558217) #서귀포시의 위도 경도

#train
train['dis_jejusi'] = [geopy.distance.vincenty((train['latitude'].iloc[i],train['longitude'].iloc[i]), coords_jejusi).km for i in range(len(train))]
train['dis_seoquipo'] = [geopy.distance.vincenty((train['latitude'].iloc[i],train['longitude'].iloc[i]), coords_seoquipo).km for i in range(len(train))]
#test
test['dis_jejusi'] = [geopy.distance.vincenty((test['latitude'].iloc[i],test['longitude'].iloc[i]), coords_jejusi).km for i in range(len(test))]
test['dis_seoquipo'] = [geopy.distance.vincenty((test['latitude'].iloc[i],test['longitude'].iloc[i]), coords_seoquipo).km for i in range(len(test))]

In [42]:
# corr 로 둘의 상관관계를 확인
print(train['dis_jejusi'].corr(train['18~20_ride']))
print(train['dis_seoquipo'].corr(train['18~20_ride']))

-0.12380548460763058
0.022087013666358277


제주시는 거리가 멀어질수록, 줄어들지만, 서귀포시는 애매하죠?

In [43]:
# 칼럼 수 확인
train.shape

(415423, 30)

컬럼이 또 2개 늘어, 30개가 되었다.

In [44]:
#우리에게 주어진 칼럼을 확인한다.
train.columns.get_values()

array(['id', 'date', 'bus_route_id', 'in_out', 'station_code',
       'station_name', 'latitude', 'longitude', '6~7_ride', '7~8_ride',
       '8~9_ride', '9~10_ride', '10~11_ride', '11~12_ride', '6~7_takeoff',
       '7~8_takeoff', '8~9_takeoff', '9~10_takeoff', '10~11_takeoff',
       '11~12_takeoff', '18~20_ride', 'weekday_0', 'weekday_1',
       'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6',
       'dis_jejusi', 'dis_seoquipo'], dtype=object)

1. 'id' : 식별자임으로 이용 불가 X

2. 'date': weekday로 변환하는데 이용됨 X

3. 'bus_route_id' : 특정 버스마다 
이용고객이 다를 것이니 이용가는 하지 않을까 ?

4. 'in_out' : 시내 시외 구분으로 사용 됨 O

5.  'station_code' : 탑승 정류장 ?

6. 'latitude', 'longitude' : 주요 도시와의 
거리를 재는데 사용 됨 X

7. '67_ride', '78_ride', '89_ride', 
'910_ride', '1011_ride', '1112_ride', '67_takeoff','78_takeoff', '89_takeoff', '910_takeoff', '1011_takeoff', '1112_takeoff' : 실제 탑승 인원 O

8. 'weekday_' : date로 사용됨

9. 'dis_jejusi', 'dis_seoquipo' : 주요 도시와의 거리 O

10. '18~20_ride' : y 값  


   , 

일단 예시를 활용해서 예측값을 구해본다.

In [0]:
input_var=['in_out','latitude','longitude','6~7_ride', '7~8_ride', '8~9_ride', '9~10_ride',
       '10~11_ride', '11~12_ride', '6~7_takeoff', '7~8_takeoff', '8~9_takeoff',
       '9~10_takeoff', '10~11_takeoff', '11~12_takeoff','weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
       'weekday_5', 'weekday_6', 'dis_jejusi', 'dis_seoquipo']
target=['18~20_ride']

In [0]:
X_train=train[input_var]
y_train=train[target]

X_test=test[input_var]

In [0]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=1217)

In [61]:
rf.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=1217,
                      verbose=0, warm_start=False)

In [0]:
test['18~20_ride'] = rf.predict(X_test)
test[['id','18~20_ride']].to_csv("dacon_base_line.csv",index=False)

In [0]:
sample = pd.read_csv("/content/drive/My Drive/데이콘 공모전/데이터/13회 제주테크노파크 버스정류장 승하차 예측 data/submission_sample.csv")

In [64]:
sample

,id,18~20_ride
0,415423,0
1,415424,0
2,415425,0
3,415426,0
4,415427,0
...,...,...
228165,643588,0
228166,643589,0
228167,643590,0
228168,643591,0
